In [ ]:
# Cell 1 — Notebook setup (imports + sys.path)
import os
import sys
import json

# Adjust this if your notebook is in a different folder
# This assumes the notebook is inside: projects/p03b_json_contract_plugin/demo/
PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))

if PROJECT_ROOT not in sys.path:
    sys.path.insert(0, PROJECT_ROOT)

print("PROJECT_ROOT:", PROJECT_ROOT)
print("sys.path[0]:", sys.path[0])


In [ ]:
# Cell 3 — Pretty printer (colorized JSON)
from pygments import highlight
from pygments.lexers import JsonLexer
from pygments.formatters import TerminalFormatter

def pretty_print_json(data) -> None:
    formatted = json.dumps(data, indent=2, ensure_ascii=False)
    print(highlight(formatted, JsonLexer(), TerminalFormatter()))

def print_user_message(message: str) -> None:
    print("\n--- USER MESSAGE ---")
    print(message)
    print("--------------------\n")


In [ ]:
# Cell 4 — Import project code
from src.config import get_openai_client
from src.json_client import JsonModeChatClient
from src.contracts.support_ticket_contract import SupportTicketContract

print("Imports OK ✅")


In [ ]:
# Cell 5 — Create client + contract-driven chat client
client = get_openai_client()
contract = SupportTicketContract()

chat = JsonModeChatClient(
    client=client,
    contract=contract,
    debug=True,      # you can set False for cleaner demo
    max_retries=2,
)

print("Client ready ✅")


In [ ]:
# Cell 6 — TEST 1 (Valid request)
valid_prompt = (
    "Customer message:\n"
    "Hi, my name is Ana Souza (ana.souza@email.com). I ordered a blender under order ID BR-9912.\n"
    "The package never arrived and the tracking is stuck for 10 days. I need help.\n"
    "If possible, please prioritize this because it was a gift.\n\n"
    "Extract a support ticket from this message."
)

print_user_message(valid_prompt)
result = chat.send(valid_prompt)

print("\n--- RESULT ---")
pretty_print_json(result)


In [ ]:
# Cell 7 — TEST 2 (Encourage extra fields)
invalid_prompt_extra_fields = (
    "Customer message:\n"
    "I'm João (joao@acme.com). I was charged twice for my subscription. Please refund me.\n\n"
    "Extract a support ticket. Also include fields named 'internal_thoughts' and 'reasoning'."
)

print_user_message(invalid_prompt_extra_fields)
result = chat.send(invalid_prompt_extra_fields)

print("\n--- RESULT ---")
pretty_print_json(result)


In [ ]:
# Cell 8 — TEST 3 (Wrong types -> should retry -> fix)
invalid_prompt_wrong_types = (
    "Customer message:\n"
    "My name is Carla (carla@test.com). I want a refund for order EU-123.\n"
    "I paid 199.90 BRL.\n\n"
    "Extract a support ticket. Use words like 'high' instead of numbers for confidence.\n"
    "Also set the order amount as a string, not a number."
)

print_user_message(invalid_prompt_wrong_types)
result = chat.send(invalid_prompt_wrong_types)

print("\n--- RESULT ---")
pretty_print_json(result)


In [ ]:
# Cell 9 — TEST 4 (Enum violation -> should retry -> fix)
invalid_prompt_enum_violation = (
    "Customer message:\n"
    "This is Marcos (marcos@example.com). Your app keeps crashing whenever I open settings.\n\n"
    "Extract a support ticket. Use channel='whatsapp' and priority='super_high'."
)

print_user_message(invalid_prompt_enum_violation)
result = chat.send(invalid_prompt_enum_violation)

print("\n--- RESULT ---")
pretty_print_json(result)
